In [1]:
from jax import numpy as np


def read_data(filename):
    labels, sentences_without_label = [], []
    label_dict = { "E": [[1, 0], [0, 0]], "P": [[0, 1], [0, 0]], "S": [[0, 0], [1, 0]], "T": [[0, 0], [0, 1]]}
    with open(filename) as f:
        data_raw = f.readlines()
        for sentence in data_raw:
            words = sentence.split()
            label = label_dict[words[-1]]
            sentence_without_label = ' '.join(words[:-1]).lower()
            labels.append(label)
            sentences_without_label.append(sentence_without_label)
    return np.array(labels), sentences_without_label


train_labels, train_data = read_data('../datasets/news_topic_train_data.txt')
print(type)
dev_labels, dev_data = read_data('../datasets/news_topic_dev_data.txt')
test_labels, test_data = read_data('../datasets/news_topic_test_data.txt')

FileNotFoundError: [Errno 2] No such file or directory: '../datasets/news_topic_train_data.txt'

In [ ]:
from lambeq.ccg2discocat import DepCCGParser

reader = DepCCGParser(possible_root_cats=['S[dcl]'])

train_diagrams = reader.sentences2diagrams(train_data)
dev_diagrams = reader.sentences2diagrams(dev_data)
test_diagrams = reader.sentences2diagrams(test_data)

In [ ]:
# Visualisation

for d in train_diagrams:
    d.draw()

In [ ]:
# from discopy.rigid import Id


# def remove_cups(diagram):
#     # Remove cups to reduce post-selection in the circuit, for faster execution

#     diags = []
#     for box, offset in zip(diagram.boxes, diagram.offsets):
#         if not box.dom:  # word box
#             diags.insert(offset, box)
#         else:  # cup (the only other type of box in these diagrams)
#             i = 0
#             off = offset
#             while off != len(diags[i].cod) - 1:
#                 assert off > 0
#                 off -= len(diags[i].cod)
#                 i += 1
#             left, right = diags[i:i+2]
            
#             if len(left.cod) == 1:
#                 new_diag = right >> (left.r.dagger() @ Id(right.cod[1:]))
#             else:
#                 assert len(right.cod) == 1
#                 new_diag = left >> (Id(left.cod[:-1]) @ right.l.dagger())

#             diags[i:i+2] = [new_diag]

#     assert len(diags) == 1
#     return diags[0]


# train_diagrams = [remove_cups(diagram) for diagram in raw_train_diagrams]
# dev_diagrams = [remove_cups(diagram) for diagram in raw_dev_diagrams]
# test_diagrams = [remove_cups(diagram) for diagram in raw_test_diagrams]

# train_diagrams[0].draw()

In [ ]:
# Simplify

from discopy import Word, grammar
from discopy.rigid import Cup, Diagram, Id
from lambeq.rewrite import Rewriter

rewriter = Rewriter()
simplified_train_diagrams = []
simplified_dev_diagrams = []
simplified_test_diagrams = []

for d in train_diagrams:
    simplified_train_diagrams.append(rewriter(d).normal_form())
    
for d in dev_diagrams:
    simplified_dev_diagrams.append(rewriter(d).normal_form())

for d in test_diagrams:
    simplified_test_diagrams.append(rewriter(d).normal_form())

In [ ]:
from lambeq.circuit import IQPAnsatz
from lambeq.core.types import AtomicType

ansatz = IQPAnsatz({AtomicType.NOUN: 1, AtomicType.SENTENCE: 2},
                   n_layers=1, n_single_qubit_params=3)

train_circuits = [ansatz(diagram) for diagram in simplified_train_diagrams]
dev_circuits =  [ansatz(diagram) for diagram in simplified_dev_diagrams]
test_circuits = [ansatz(diagram) for diagram in simplified_test_diagrams]

train_circuits[0].draw(figsize=(9, 12))

In [ ]:
from sympy import default_sort_key

all_circuits = train_circuits + dev_circuits + test_circuits

# sort the symbols since they are returned as a set
parameters = sorted(
    {s for circ in all_circuits for s in circ.free_symbols},
    key=default_sort_key)

In [ ]:
from discopy.quantum import Circuit
from discopy.tensor import Tensor
from jax import jit

Tensor.np = np


def normalise(predictions):
    # apply smoothing to predictions
    predictions = np.abs(predictions) + 1e-9
    return predictions / predictions.sum()


def make_pred_fn(circuits):
    circuit_fns = [c.lambdify(*parameters) for c in circuits]

    def predict(params):
        outputs = Circuit.eval(*(c(*params) for c in circuit_fns))
        return np.array([normalise(output.array).flatten() for output in outputs])
    return predict


train_pred_fn = jit(make_pred_fn(train_circuits))
dev_pred_fn = jit(make_pred_fn(dev_circuits))
test_pred_fn = make_pred_fn(test_circuits)

In [ ]:
# from noisyopt import minimizeSPSA
# import numpy


# def make_cost_fn(pred_fn, labels):
#     def cost_fn(params, **kwargs):
#         predictions = pred_fn(params)
#         cost = -np.sum(labels * np.log(predictions)) / len(labels)  # cross-entropy loss
#         costs.append(cost) 
        
#         rounded_preds = []
#         for pred in predictions:
#             rounded_pred = [1 if i == np.argmax(pred) else 0 for i in range(len(pred))]
#             rounded_preds.append(rounded_pred)
#         rounded_preds = np.array(rounded_preds)  

        
#         acc = np.sum(rounded_preds == labels) / len(labels) / 4  # half due to double-counting
#         accuracies.append(acc)

#         return cost
    
#     costs, accuracies = [], []
#     return cost_fn, costs, accuracies


# train_cost_fn, train_costs, train_accs = make_cost_fn(train_pred_fn, train_labels)
# dev_cost_fn, dev_costs, dev_accs = make_cost_fn(dev_pred_fn, dev_labels)

# SEED = 0
# rng = numpy.random.default_rng(SEED)
# x0 = np.array(rng.random(len(parameters)))
# numpy.random.seed(SEED)
# result = minimizeSPSA(train_cost_fn, x0=x0, a=0.2, c=0.06, niter=15000, callback=dev_cost_fn)

In [ ]:
# import matplotlib.pyplot as plt

# fig, ((ax_tl, ax_tr), (ax_bl, ax_br)) = plt.subplots(2, 2, sharex=True, sharey='row', figsize=(10, 6))
# ax_tl.set_title('Training set')
# ax_tr.set_title('Development set')
# ax_bl.set_xlabel('Iterations')
# ax_br.set_xlabel('Iterations')
# ax_bl.set_ylabel('Accuracy')
# ax_tl.set_ylabel('Loss')

# colours = iter(plt.rcParams['axes.prop_cycle'].by_key()['color'])
# ax_tl.plot(train_costs[1::2], color=next(colours))  # training evaluates twice per iteration
# ax_bl.plot(train_accs[1::2], color=next(colours))   # so take every other entry
# ax_tr.plot(dev_costs, color=next(colours))
# ax_br.plot(dev_accs, color=next(colours))

# # print test accuracy
# test_cost_fn, _, test_accs = make_cost_fn(test_pred_fn, test_labels)
# test_cost_fn(result.x)
# print('Test accuracy:', test_accs[0])

In [ ]:
def get_prediction(sentence):
    diagram = rewriter(reader.sentence2diagram(sentence)).normal_form()
    circuit = ansatz(diagram)
    circuit_fn = circuit.lambdify(*parameters)

    output = Circuit.eval(circuit_fn(*result.x))
    return np.array(normalise(output.array).flatten())

In [ ]:
from nltk import CFG
from nltk.parse.generate import generate
import pickle
from lambeq.circuit import IQPAnsatz
from lambeq.core.types import AtomicType

NEWS_GRAMMAR = """
  S -> NP VP
  NP -> N
  PP -> P NP
  NP -> A N
  A -> 'new' | 'expensive' | 'interesting' | 'prestigious'
  VP -> 'beats' NP | 'continues' NP | 'continues' PP | 'comeback' PP | 'unveils' NP | 'helps' NP | 'wins' NP | 'defends' NP | 'attacks' NP |
  N -> 'xbox' | 'microsoft' | 'sony' | 'italy' | 'broadband' | 'england' | 'playstation' | 'arsenal' | 'chelsea' | 'labour' | 'economy'| 'johnson' | 'policy' | 'final' | 'film' | 'award' | 'spider-man' | 'director' | 'meryl streep'
  P -> 'in' | 'on' | 'to' | 'for'
"""

GRAMMAR = CFG.fromstring(NEWS_GRAMMAR)
SENTENCES = list(generate(GRAMMAR, depth=5))

VOCAB = set()

RESULT = pickle.load(open('result.pickle', 'rb'))
PARAMETERS = pickle.load(open('parameters.pickle', 'rb'))

print(PARAMETERS)
print(RESULT)

ANSATZ = IQPAnsatz({AtomicType.NOUN: 1, AtomicType.SENTENCE: 2},
                   n_layers=1, n_single_qubit_params=3)

READER = DepCCGParser(possible_root_cats=['S[dcl]'])


for p in PARAMETERS:
    split_string = str(p).split("_", 1)
    VOCAB.add(split_string[0])
    
VOCAB = list(VOCAB)
    
def _normalise(predictions):
    # apply smoothing to predictions
    predictions = np.abs(predictions) + 1e-9
    return predictions / predictions.sum()

def _get_prediction(sentence):
    diagram = rewriter(reader.sentence2diagram(sentence)).normal_form()
    circuit = ansatz(diagram)
    circuit_fn = circuit.lambdify(*PARAMETERS)
#     output = circuit_fn(*params).eval()

    output = Circuit.eval(circuit_fn(*RESULT.x))
    return np.array(_normalise(output.array).flatten())



In [ ]:
# import pickle 
# pickle.dump(result, open('result.pickle', 'wb'))
# pickle.dump(parameters, open('parameters.pickle', 'wb'))

In [ ]:
# def get_sentence_with_topic(topic):
#     rewriter = Rewriter()
#     for i in range(1000):
#         print (i)
#         try:
#             sentence = " ".join(random.choice(SENTENCES))
#             diagram = rewriter(READER.sentence2diagram(sentence)).normal_form()
#             circuit = ANSATZ(diagram)
#             prediction = _get_prediction(circuit)
#             print(prediction)
#         except:
#             continue
#         if topic == "E" and prediction[0] > 0.8:
#             return sentence
#             break
#         elif topic == "P" and prediction[1] > 0.8:
#             return sentence
#             break
#         elif topic == "S" and prediction[2] > 0.8:
#             return sentence
#             break
#         elif topic == "T" and prediction[3] > 0.8:
#             return sentence
#             break

In [ ]:
# import random
# from math import exp
# print(type(SENTENCES[5][0]))
# topic = ''
# while (topic != 'E' and topic != 'S' and topic != 'P' and topic != 'T'):
#     print("What headlines do you wanna know about? Tech (T), Entertainment (E), Sports(S), or Politics(P)?")
#     topic = input()

# # objective function
# def objective(words):
#     try:
#         sentence = " ".join(words)
#         diagram = rewriter(reader.sentence2diagram(sentence)).normal_form()
#         circuit = ansatz(diagram)
#         prediction = _get_prediction(sentence)
#     except:
#         return 0

#     if topic == "E":
#         return prediction[0]
#     elif topic == "P":
#         return prediction[1]
#     elif topic == "S":
#         return prediction[2]
#     elif topic == "T":
#         return prediction[3]


 
# #### simulated annealing algorithm
# # generate an initial point

# curr = random.choice(SENTENCES)
# print (curr in SENTENCES)

# print("initial_sentence: {}".format(" ".join(curr)))
# # evaluate the initial point
# curr_eval = objective(curr)
# print("initial_eval: {}".format(curr_eval))
# if curr_eval > 0.8:
#     print("Done")
#     exit()
# temp = 5
# i = 0
# # run the algorithm

# while i < 1000:
#     i += 1
#     candidate = []
#     print("curr: {}".format(" ".join(curr)))
#     while candidate not in SENTENCES:
#         candidate = curr.copy()
#         # Randomly decide on an operation: 0 = DELETE, 1 = Replace, 2 = INSERT
#         if len(candidate) == 1:
#             op = random.randint(1, 2)
#         elif len(candidate) < 6:
#             op = random.randint(0, 2)
#         else:
#             op = random.randint(0, 1)
#         if op == 0:
#             # Remove random word
#             word = random.choice(candidate)
#             candidate.remove(word)
#         if op == 1:
#             new_word = random.choice(VOCAB)
#             index = random.randint(0,len(candidate) - 1)
#             candidate[index] = new_word
#         if op == 2:
#             # Add random word at random position
#             new_word = random.choice(VOCAB)
#             candidate.insert(random.randint(0,len(candidate)),new_word)
        
#     print("new candidate: {}".format(" ".join(candidate)))
#     candidate_eval = objective(candidate)
#     print("candidate_eval: {}".format(candidate_eval))
#     # check winning criteria
#     if candidate_eval > 0.85:
#         break
#     # difference between candidate and current point evaluation
#     diff = candidate_eval - curr_eval
#     # calculate temperature for current epoch
#     t = temp / float(i + 1)
#     # calculate metropolis acceptance criterion
#     metropolis = exp(diff / t)
#     # check if we should keep the new point
#     random_float = random.uniform(0,1)
#     if diff > 0 or (random.uniform(0, 1) < metropolis):
#     # store the new current point
#         print("SWITCHING CURR")
#         curr, curr_eval = candidate, candidate_eval
#     print("End of iteration: {}".format(i))


# print("DONE WITH SIMULATED ANNEALING")

# # print ("Trying random")

In [ ]:
def randomly_generate_sentence_with_topic(topic):
    i = 0
    while i < 200:
        i += 1
        print(i)
        try:
            sentence = " ".join(random.choice(SENTENCES))
            print(sentence)
            prediction = _get_prediction(sentence)
            print(prediction)
            if topic == "E" and prediction[0] > 0.8:
                return i, sentence
                break
            elif topic == "P" and prediction[1] > 0.8:
                return i, sentence
                break
            elif topic == "S" and prediction[0] > 0.8:
                return i, sentence
                break
            elif topic == "T" and prediction[1] > 0.8:
                return i, sentence
                break
        except:
            continue 
    return 500, "Max iterations reached" 

In [ ]:
import random
from math import exp

# objective function
def objective(words):
    try:
        sentence = " ".join(words)
        prediction = _get_prediction(sentence)
    except:
        return 0

    if topic == "E":
        return prediction[0]
    elif topic == "P":
        return prediction[1]
    elif topic == "S":
        return prediction[2]
    elif topic == "T":
        return prediction[3]

def SA_generate_sentence_with_topic(topic):
#### simulated annealing algorithm

    # generate an initial point
    curr = random.choice(SENTENCES)
    print("initial_sentence: {}".format(" ".join(curr)))
    # evaluate the initial point
    curr_eval = objective(curr)
    print("initial_eval: {}".format(curr_eval))
    if curr_eval > 0.85:
        return 1, " ".join(curr)

    temp = 5
    i = 0
    # run the algorithm
    while i < 500:
        i += 1
        candidate = []
        print("curr: {}".format(" ".join(curr)))
        while candidate not in SENTENCES:
            candidate = curr.copy()
            # Randomly decide on an operation: 0 = DELETE, 1 = Replace, 2 = INSERT
            if len(candidate) == 1:
                op = random.randint(1, 2)
            elif len(candidate) < 6:
                op = random.randint(0, 2)
            else:
                op = random.randint(0, 1)
            if op == 0:
                # Remove random word
                word = random.choice(candidate)
                candidate.remove(word)
            if op == 1:
                new_word = random.choice(VOCAB)
                index = random.randint(0,len(candidate) - 1)
                candidate[index] = new_word
            if op == 2:
                # Add random word at random position
                new_word = random.choice(VOCAB)
                candidate.insert(random.randint(0,len(candidate)),new_word)

        print("new candidate: {}".format(" ".join(candidate)))
        candidate_eval = objective(candidate)
        print("candidate_eval: {}".format(candidate_eval))
        # check winning criteria
        if candidate_eval > 0.8:
            return i, " ".join(candidate)
        # difference between candidate and current point evaluation
        diff = candidate_eval - curr_eval
        # calculate temperature for current epoch
        t = temp / float(i + 1)
        # calculate metropolis acceptance criterion
        metropolis = exp(diff / t)
        # check if we should keep the new point
        random_float = random.uniform(0,1)
        if diff > 0 or (random.uniform(0, 1) < metropolis):
        # store the new current point
            print("SWITCHING CURR")
            curr, curr_eval = candidate, candidate_eval
        print("End of iteration: {}".format(i))
    return 500, "Max iterations reached"

In [ ]:
import random
from math import exp
import time

topic = 'P'


# t1 = time.perf_counter()
# SA_gen_results = []
# while len(SA_gen_results) < 30:
#     print("SA Generating sentence {}".format(len(SA_gen_results)))
#     iterations, sentence = SA_generate_sentence_with_topic(topic)
#     if (sentence not in [r[1] for r in SA_gen_results]) or (sentence == "Max iterations reached"):
#         SA_gen_results.append((iterations, sentence))

# t2 = time.perf_counter()
# total_time = t2 - t1
    
# with open("news_simulated_annealing_generation_results_politics", "w") as f:
#     f.write("total_time: {} \n".format(total_time))
#     for iterations, sentence in SA_gen_results:
#         f.write("Generated sentence: {} --- Number of iterations: {} \n".format(sentence, iterations))      


t1 = time.perf_counter()
random_gen_results = []
while len(random_gen_results) < 30:
    print("Randomly Generating sentence {}".format(len(random_gen_results)))
    iterations, sentence = randomly_generate_sentence_with_topic(topic)
    if (sentence not in [r[1] for r in random_gen_results]) or (sentence == "Max iterations reached"):
        random_gen_results.append((iterations, sentence))

t2 = time.perf_counter()
total_time = t2 - t1
    
with open("news_random_generation_results_politics", "w") as f:
    f.write("total_time: {} \n".format(total_time))
    for iterations, sentence in random_gen_results:
        f.write("Generated sentence: {} --- Number of iterations: {} \n".format(sentence, iterations))  

In [ ]:
print(SENTENCES)